In [1]:
!pip install catboost

     ---------------------------------------- 74.0/74.0 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 15.3/15.3 MB 2.5 MB/s eta 0:00:00


#### 참고 블로그 
https://jimmy-ai.tistory.com/178
<br>
https://ekfkdlxm.tistory.com/37

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from IPython.display import Image
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import KFold
#from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
#from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family = 'D2coding')
mpl.rc('axes', unicode_minus=False)

sns.set(font="malgun gothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10, 8))

warnings.filterwarnings('ignore')

In [13]:
data = pd.read_csv('C:/k_digital/source/data/open/train.csv')
data1 = pd.read_csv('C:/k_digital/source/data/open/train.csv')
test = pd.read_csv('C:/k_digital/source/data/open/test.csv')
data

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [14]:
data_features = ['YEAR','COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 
                       'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

data = data[data_features]
# test = test[data_features]
# 'COMPONENT_ARBITRARY', 'ANONYMOUS_1'

In [15]:
test_features = ['YEAR','COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 
                       'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
test = test[test_features]

In [16]:
data.shape

(14095, 18)

In [17]:
data.head(3)

,YEAR,COMPONENT_ARBITRARY,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2011,COMPONENT3,1486,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75
1,2021,COMPONENT2,1350,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652
2,2015,COMPONENT2,2415,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412


# 라벨인코딩

In [18]:
le = LabelEncoder()
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
for col in categorical_features:    
    data[col] = le.fit_transform(data[col])

In [19]:
le = LabelEncoder()
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
for col in categorical_features:    
    test[col] = le.fit_transform(test[col])

In [20]:
data['YEAR'].unique()

array([ 4, 14,  8,  3,  1,  2,  7, 11,  6,  9, 10,  5, 12,  0, 15, 13],
      dtype=int64)

In [21]:
data.head(3)

,YEAR,COMPONENT_ARBITRARY,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,4,2,1486,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75
1,14,1,1350,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652
2,8,1,2415,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412


In [22]:
# 스케일링
data_need_scale = data.drop(columns = ['COMPONENT_ARBITRARY', 'YEAR'])

ss = StandardScaler()
data_scaled = ss.fit_transform(data_need_scale)
data_scaled

array([[-0.39376339, -0.3407601 , -0.15021362, ..., -0.1065502 ,
         0.89989172, -0.96600175],
       [-0.42602191, -0.0225763 , -0.15021362, ..., -0.1065502 ,
        -1.31737552,  0.11914661],
       [-0.17340923, -0.3407601 , -0.15021362, ..., -0.1065502 ,
        -0.74088603, -0.33221493],
       ...,
       [-0.3221305 ,  0.29560751, -0.15021362, ..., -0.1065502 ,
        -1.11580577,  1.33970343],
       [-0.15372204,  0.29560751, -0.15021362, ..., -0.1065502 ,
        -1.31132842,  0.11914661],
       [-0.29509027, -0.3407601 , -0.15021362, ..., -0.1065502 ,
        -1.25690459,  0.04391969]])

In [23]:
test_need_scale = test.drop(columns = ['COMPONENT_ARBITRARY', 'YEAR'])

ss = StandardScaler()
test_scaled = ss.fit_transform(test_need_scale)
test_scaled

array([[-0.20832585, -0.34182023, -0.14900964, ..., -0.12384153,
        -0.42300603,  0.96054395],
       [-0.08380289, -0.34182023, -0.14900964, ..., -0.12384153,
         0.36151644, -1.0724087 ],
       [-0.2517641 , -0.34182023, -0.14900964, ..., -0.12384153,
        -1.45875198,  0.25023519],
       ...,
       [ 0.23288258, -0.34182023, -0.14900964, ..., -0.12384153,
         4.22462848, -1.07052459],
       [-0.37959664, -0.34182023, -0.14900964, ..., -0.12384153,
        -1.30889938, -0.19441432],
       [-0.16592186,  2.13663024,  5.40757816, ..., -0.12384153,
        -0.6720258 ,  0.73256687]])

In [24]:
data_need_scale.columns

Index(['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN'],
      dtype='object')

In [25]:
data_name = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']


data_df = pd.DataFrame(data_scaled, columns = data_name)
data_df

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,-0.393763,-0.340760,-0.150214,-0.089633,0.339245,0.336858,1.331290,-0.041588,1.186914,-0.384284,1.384414,5.293270,0.622282,-0.10655,0.899892,-0.966002
1,-0.426022,-0.022576,-0.150214,-0.089633,-0.115388,-0.027612,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.259244,-0.102635,-0.10655,-1.317376,0.119147
2,-0.173409,-0.340760,-0.150214,-0.089633,-0.080416,-0.252497,-0.326655,-0.041588,-0.250456,-0.400998,-0.191804,-0.260552,-0.102635,-0.10655,-0.740886,-0.332215
3,1.006399,-0.340760,-0.150214,-0.089633,-0.115388,-0.260252,-0.264764,-0.041588,-0.160621,-0.400998,-0.191804,-0.242884,-0.102635,-0.10655,0.482642,-1.093888
4,0.191634,-0.340760,-0.150214,-0.089633,-0.115388,-0.268007,-0.200996,-0.041588,-0.250456,-0.400998,-0.191804,-0.129674,-0.102635,-0.10655,0.478611,-0.866326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,-0.362928,-0.340760,-0.150214,-0.089633,-0.115388,-0.244743,-0.291021,-0.041588,-0.250456,-0.400998,-0.191804,-0.248773,-0.102635,-0.10655,0.524972,-1.076961
14091,-0.085884,-0.340760,-0.150214,-0.089633,-0.115388,-0.252497,-0.311651,-0.041588,-0.250456,3.342831,-0.191804,-0.265787,-0.102635,-0.10655,0.164162,1.540935
14092,-0.322130,0.295608,-0.150214,-0.089633,0.024499,-0.213724,0.444177,-0.041588,0.378393,-0.233862,0.070899,0.150406,-0.102635,-0.10655,-1.115806,1.339703
14093,-0.153722,0.295608,-0.150214,-0.089633,-0.115388,1.050289,-0.298523,-0.041588,-0.250456,-0.384284,-0.191804,-0.264479,-0.102635,-0.10655,-1.311328,0.119147


In [26]:
test_name = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']


test_df = pd.DataFrame(test_scaled, columns = data_name)
test_df

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,-0.208326,-0.34182,-0.149010,-0.114836,-0.149412,-0.236485,-0.270094,-0.073334,-0.276403,-0.402347,-0.154766,-0.256602,-0.108232,-0.123842,-0.423006,0.960544
1,-0.083803,-0.34182,-0.149010,-0.114836,-0.054711,-0.236485,0.114896,-0.073334,-0.011265,-0.402347,-0.154766,1.245189,0.027530,-0.123842,0.361516,-1.072409
2,-0.251764,-0.34182,-0.149010,-0.114836,-0.149412,-0.139025,-0.280225,-0.073334,-0.276403,-0.402347,-0.154766,-0.256050,-0.108232,-0.123842,-1.458752,0.250235
3,-0.371323,-0.34182,-0.149010,-0.114836,-0.007360,-0.216993,-0.051547,-0.073334,0.077114,-0.354146,-0.154766,4.155529,-0.108232,-0.123842,0.711907,-0.917912
4,1.039593,-0.34182,-0.149010,-0.114836,-0.149412,-0.203999,-0.268646,-0.073334,-0.276403,-0.402347,-0.154766,-0.253291,-0.108232,-0.123842,-1.037842,-0.211371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,-0.307200,-0.34182,-0.149010,-0.114836,-0.007360,0.601670,1.227891,-0.073334,5.468260,-0.322013,0.195596,0.840776,-0.108232,-0.123842,-0.865953,1.096200
6037,0.192754,-0.34182,-0.149010,-0.114836,0.087341,-0.229988,0.777772,-0.073334,0.165494,-0.402347,-0.154766,0.120225,-0.108232,-0.123842,0.601721,-1.070525
6038,0.232883,-0.34182,-0.149010,-0.114836,-0.149412,-0.242983,-0.210753,-0.073334,-0.276403,-0.402347,-0.154766,-0.224050,-0.108232,-0.123842,4.224628,-1.070525
6039,-0.379597,-0.34182,-0.149010,-0.114836,-0.149412,0.159852,-0.284567,-0.073334,-0.276403,-0.402347,-0.154766,-0.259360,-0.108232,-0.123842,-1.308899,-0.194414


In [27]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
data_label = data[categorical_features]
data_new = pd.concat([data_label,data_df], axis=1)

In [28]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
test_label = test[categorical_features]
test_new = pd.concat([test_label,test_df], axis=1)

In [29]:
test_new.head(3)

,COMPONENT_ARBITRARY,YEAR,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,0,9,-0.208326,-0.34182,-0.14901,-0.114836,-0.149412,-0.236485,-0.270094,-0.073334,-0.276403,-0.402347,-0.154766,-0.256602,-0.108232,-0.123842,-0.423006,0.960544
1,2,4,-0.083803,-0.34182,-0.14901,-0.114836,-0.054711,-0.236485,0.114896,-0.073334,-0.011265,-0.402347,-0.154766,1.245189,0.027530,-0.123842,0.361516,-1.072409
2,1,3,-0.251764,-0.34182,-0.14901,-0.114836,-0.149412,-0.139025,-0.280225,-0.073334,-0.276403,-0.402347,-0.154766,-0.256050,-0.108232,-0.123842,-1.458752,0.250235


In [30]:
data_new.head(3)

,COMPONENT_ARBITRARY,YEAR,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2,4,-0.393763,-0.340760,-0.150214,-0.089633,0.339245,0.336858,1.331290,-0.041588,1.186914,-0.384284,1.384414,5.293270,0.622282,-0.10655,0.899892,-0.966002
1,1,14,-0.426022,-0.022576,-0.150214,-0.089633,-0.115388,-0.027612,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.259244,-0.102635,-0.10655,-1.317376,0.119147
2,1,8,-0.173409,-0.340760,-0.150214,-0.089633,-0.080416,-0.252497,-0.326655,-0.041588,-0.250456,-0.400998,-0.191804,-0.260552,-0.102635,-0.10655,-0.740886,-0.332215


In [31]:
answer = data1['Y_LABEL'] 

## train_test_split

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_new, answer, test_size = 0.2, stratify = answer, random_state = 2022)

# voting

In [33]:
# 보팅 분류기 Voting
# 3개 
from sklearn.ensemble import VotingClassifier

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=3)
svc = SVC(C= 0.1, gamma='auto', probability=True) # C값을 바꿔주면서 테스트해야할듯 
dt = DecisionTreeClassifier(max_depth=3)

estimators = [('knn', knn), ('dt', dt), ('svc', svc)]  # 모델 집합

voting_model = VotingClassifier(estimators, voting = 'soft')

voting_model.fit(X_train, y_train)
pred = voting_model.predict(X_test)
pred_test = voting_model.predict(test_new) # 우리가 제출할 값 


models = [knn, dt, svc]

# train 데이터만 활용했을 때 f1 스코어 
for model in models :
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    model_name = model.__class__.__name__
    print('%s 정확도 : %.3f' %(model_name, accuracy_score(pred, y_test)))
    print('Macro f1 socre : ', f1_score(y_test, pred, labels=[0,1], average='macro'))

KNeighborsClassifier 정확도 : 0.903
Macro f1 socre :  0.5265936582184763
DecisionTreeClassifier 정확도 : 0.914
Macro f1 socre :  0.4816660284347514
SVC 정확도 : 0.915
Macro f1 socre :  0.4776727811747267


# Kfold

In [34]:
from sklearn.model_selection import KFold

X = np.array(data_new)
y = answer
acc_mean = []
n_iter = 0

# split 개수, 셔플 여부 및 seed 설정
kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # 학습을 진행하겠다면?
    voting_model.fit(X_train, y_train)  # voting 모델로 검증
    # 예측
    fold_pred = voting_model.predict(X_test)
    pred_test = voting_model.predict(test_new)
     # 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 4)
    print('\n{} 교차검증  : {} , 학습 데이터 : {} , 검증 데이터 : {}'.format(n_iter, accuracy, X_train.shape[0],
                                                                X_test.shape[0]))
    print('Macro f1 socre : ', f1_score(y_test,fold_pred, labels=[0,1], average='macro'))

    acc_mean.append(accuracy)
print('\n')

print('\n 평균 정확도 :  ', np.mean(acc_mean))  # 평균정확도


1 교차검증  : 0.9159 , 학습 데이터 : 11276 , 검증 데이터 : 2819
Macro f1 socre :  0.4780596185891502

2 교차검증  : 0.9134 , 학습 데이터 : 11276 , 검증 데이터 : 2819
Macro f1 socre :  0.4932216976405614

3 교차검증  : 0.9145 , 학습 데이터 : 11276 , 검증 데이터 : 2819
Macro f1 socre :  0.4897936807259362

4 교차검증  : 0.911 , 학습 데이터 : 11276 , 검증 데이터 : 2819
Macro f1 socre :  0.49581693562121054

5 교차검증  : 0.9234 , 학습 데이터 : 11276 , 검증 데이터 : 2819
Macro f1 socre :  0.5021682397033141



 평균 정확도 :   0.91564


# 오차행렬
- 분류 문제에서 오차행렬 반드시 만들라고 한다.
- https://velog.io/@ljs7463/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EB%AA%A8%EB%8D%B8-%ED%8F%89%EA%B0%80%EC%A0%95%EB%B0%80%EB%8F%84%EC%9E%AC%ED%98%84%EC%9C%A8f1-score%EB%93%B1 

In [42]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(y_test, fold_pred), '\n\n') # negative라고 예측했는데 실제 negative인 친구들 매우 많다. (?)
print(classification_report(y_test, fold_pred))

[[2598    0]
 [ 216    5]] 


              precision    recall  f1-score   support

           0       0.92      1.00      0.96      2598
           1       1.00      0.02      0.04       221

    accuracy                           0.92      2819
   macro avg       0.96      0.51      0.50      2819
weighted avg       0.93      0.92      0.89      2819



In [36]:
#sub = pd.read_csv('voting(3+kfold)_submission.csv')

In [37]:
#sub['Y_LABEL'] = pred_test
#sub.head(3)

In [38]:
#sub.to_csv('voting(3+kfold)_submission',index=False)

# TEST

In [39]:
test_new

,COMPONENT_ARBITRARY,YEAR,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,0,9,-0.208326,-0.34182,-0.149010,-0.114836,-0.149412,-0.236485,-0.270094,-0.073334,-0.276403,-0.402347,-0.154766,-0.256602,-0.108232,-0.123842,-0.423006,0.960544
1,2,4,-0.083803,-0.34182,-0.149010,-0.114836,-0.054711,-0.236485,0.114896,-0.073334,-0.011265,-0.402347,-0.154766,1.245189,0.027530,-0.123842,0.361516,-1.072409
2,1,3,-0.251764,-0.34182,-0.149010,-0.114836,-0.149412,-0.139025,-0.280225,-0.073334,-0.276403,-0.402347,-0.154766,-0.256050,-0.108232,-0.123842,-1.458752,0.250235
3,2,2,-0.371323,-0.34182,-0.149010,-0.114836,-0.007360,-0.216993,-0.051547,-0.073334,0.077114,-0.354146,-0.154766,4.155529,-0.108232,-0.123842,0.711907,-0.917912
4,1,6,1.039593,-0.34182,-0.149010,-0.114836,-0.149412,-0.203999,-0.268646,-0.073334,-0.276403,-0.402347,-0.154766,-0.253291,-0.108232,-0.123842,-1.037842,-0.211371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,2,7,-0.307200,-0.34182,-0.149010,-0.114836,-0.007360,0.601670,1.227891,-0.073334,5.468260,-0.322013,0.195596,0.840776,-0.108232,-0.123842,-0.865953,1.096200
6037,2,9,0.192754,-0.34182,-0.149010,-0.114836,0.087341,-0.229988,0.777772,-0.073334,0.165494,-0.402347,-0.154766,0.120225,-0.108232,-0.123842,0.601721,-1.070525
6038,2,7,0.232883,-0.34182,-0.149010,-0.114836,-0.149412,-0.242983,-0.210753,-0.073334,-0.276403,-0.402347,-0.154766,-0.224050,-0.108232,-0.123842,4.224628,-1.070525
6039,1,6,-0.379597,-0.34182,-0.149010,-0.114836,-0.149412,0.159852,-0.284567,-0.073334,-0.276403,-0.402347,-0.154766,-0.259360,-0.108232,-0.123842,-1.308899,-0.194414
